In [194]:
import pandas as pd
import numpy as np
import seaborn as sns
from pathlib import Path as P
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['KaiTi', 'SimHei', 'FangSong']  # 汉字字体,优先使用楷体，如果找不到楷体，则使用黑体
plt.rcParams['font.size'] = 12  # 字体大小
plt.rcParams['axes.unicode_minus'] = False  # 正常显示负号
from collections import defaultdict
import plotly.express as px
import pickle
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

import tensorflow.keras as keras

In [195]:
seqlen=3

In [196]:
scaler=pickle.load(P('问题2-MinMaxScaler-训练数据.csv').open('rb'))
# df=pd.read_csv('问题2-训练数据-归一化.csv',index_col='time', parse_dates=True)
df=pd.read_csv('问题2-训练数据-归一化.csv').drop(columns='time')
model=keras.models.load_model('./problem2_checkpoint.h5')

In [197]:
scaler.scale_, scaler.min_

(array([0.08025682, 0.03125977, 0.0195733 , 0.0256476 , 0.00112134,
        0.02069108]),
 array([-0.77367576, -0.92872773, -0.82912507, -0.00692485,  0.        ,
         0.48272295]))

In [198]:
df_raw=pd.read_csv('问题2-原始训练数据.csv')
df_raw

,time,10cm湿度(kg/m2),40cm湿度(kg/m2),100cm湿度(kg/m2),土壤蒸发量(mm),降水量(mm),平均气温(℃)
0,2012-01-01,13.73,30.87,42.79,0.61,2.29,-22.94
1,2012-02-01,13.00,30.87,42.79,0.86,2.29,-17.86
2,2012-03-01,12.60,30.87,42.79,1.91,21.84,-7.11
3,2012-04-01,11.97,30.73,42.76,7.20,6.10,6.94
4,2012-05-01,14.18,29.99,42.58,18.10,50.80,14.61
...,...,...,...,...,...,...,...
118,2021-11-01,13.91,52.17,93.43,1.44,530.86,-5.89
119,2021-12-01,13.14,52.14,93.45,0.93,193.80,-12.41
120,2022-01-01,12.45,52.14,93.45,0.44,607.31,-17.09
121,2022-02-01,12.10,52.14,93.45,0.27,796.04,-17.93


In [199]:
df_test = df.iloc[-seqlen:, :]
test=df_test.values.reshape([1,seqlen,-1])
df_test.shape, test.shape

((3, 6), (1, 3, 6))

In [200]:
def predict(model, df_test: pd.DataFrame, n: int, seqlen: int):
    for i in range(n):
        test=df_test.iloc[-seqlen:, :].values.reshape([1,seqlen,-1])
        pred=model.predict(test)
        df_pred=pd.DataFrame(pred, columns=df.columns)
        df_test = pd.concat([df_test, df_pred])
        
    df_test.index=pd.Index(range(len(df_test)))
    
    return df_test

In [201]:
df_pred=predict(model, df_test, 21, seqlen=seqlen)

In [202]:
pred=scaler.inverse_transform(df_pred)
df_pred=pd.DataFrame(data=pred, columns=df_pred.columns)
df_pred

,10cm湿度(kg/m2),40cm湿度(kg/m2),100cm湿度(kg/m2),土壤蒸发量(mm),降水量(mm),平均气温(℃)
0,12.450000,52.140000,93.450000,0.440000,607.310000,-17.090000
1,12.100000,52.140000,93.450000,0.270000,796.040000,-17.930000
2,14.960000,52.130000,93.440000,5.980000,115.570000,-2.360000
3,12.543921,59.673877,85.016336,-1.695563,289.498405,-18.113704
4,13.480292,59.595899,84.320882,-0.416096,243.911870,-15.596735
5,15.067871,60.343100,83.707746,2.871916,129.182480,-11.681959
6,14.197319,60.643808,81.190071,-1.107586,190.326436,-15.269624
7,15.236892,60.823220,80.206818,1.451323,162.600361,-12.228138
8,15.923503,60.954546,78.848908,3.368093,144.582336,-9.875608
9,15.680838,60.594173,77.367677,2.879606,158.291365,-9.573211


In [203]:
px.line(df_pred)